In [6]:
import pandas as pd
import os
import re

def process_excel_file(excel_filename="IntGenbyFuel2023.xlsx"):
    """
    Processes a single Excel file with 12 monthly worksheets, creates a 
    time-series DataFrame, and saves it to CSV.
    """
    
    # 1. Define the order of sheets to process
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # This regex pattern strictly matches time formats like '0:15', '1:00', or '24:00'
    time_pattern = re.compile(r'^\d{1,2}:\d{2}$')
    
    # These are the ID columns that we will not melt
    id_vars = ['Date', 'Fuel', 'Settlement Type', 'Total']
    
    all_monthly_dfs = []

    # 2. Read the entire Excel file into a dictionary of DataFrames
    print(f"Reading all worksheets from {excel_filename}...")
    try:
        # sheet_name=None reads all sheets.
        all_sheets = pd.read_excel(excel_filename, sheet_name=None)
    except FileNotFoundError:
        print(f"Error: File not found: {excel_filename}")
        print("Please upload the single .xlsx file to run this script.")
        return
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return

    # 3. Loop, Read, and Melt
    print(f"Processing 12 monthly worksheets...")
    for month_name in month_names:
        if month_name not in all_sheets:
            print(f"Warning: Worksheet '{month_name}' not found in the Excel file. Skipping.")
            continue
        
        try:
            df = all_sheets[month_name]
            
            # Identify time columns *for this specific worksheet*
            potential_time_cols = [col for col in df.columns if col not in id_vars]
            time_cols = [col for col in potential_time_cols if time_pattern.match(str(col))]
            
            if not time_cols:
                print(f"Warning: No valid time columns found in sheet '{month_name}'. Skipping.")
                continue

            # Melt the DataFrame from wide to long format
            df_long = df.melt(
                id_vars=['Date', 'Fuel'], 
                value_vars=time_cols, 
                var_name='Time',         # New column for '0:15', '0:30', etc.
                value_name='Generation_MW' # New column for the values
            )
            all_monthly_dfs.append(df_long)
            
        except Exception as e:
            print(f"Error processing sheet '{month_name}': {e}")
            
    if not all_monthly_dfs:
        print("No data was successfully processed from any worksheet. Exiting.")
        return

    # 4. Combine
    print("Combining data...")
    combined_df = pd.concat(all_monthly_dfs, ignore_index=True)

    # 5. Clean Generation_MW column
    combined_df['Generation_MW'] = pd.to_numeric(combined_df['Generation_MW'], errors='coerce')
    combined_df = combined_df.dropna(subset=['Generation_MW'])

    # 6. Create Datetime Index
    print("Creating datetime index...")
    
    # Ensure the 'Time' column is a string before doing string operations
    combined_df['Time'] = combined_df['Time'].astype(str)
    
    # Store which rows are '24:00' (end of day)
    is_2400 = (combined_df['Time'] == '24:00')
    
    # Replace '24:00' with '0:00' to make parsing uniform
    time_adj = combined_df['Time'].str.replace('24:00', '0:00')
    
    # Convert 'Date' column to datetime, then back to string to ensure 'YYYY-MM-DD' format
    date_str = pd.to_datetime(combined_df['Date']).dt.strftime('%Y-%m-%d')
    
    # Create the full datetime string (e.g., '2023-01-01 0:15')
    datetime_str = date_str + ' ' + time_adj
    
    # Convert to datetime. This is the *end* of the interval.
    interval_end_dt = pd.to_datetime(datetime_str, format='%Y-%m-%d %H:%M', errors='coerce')
    
    # Add 1 day to all the rows that were originally '24:00'
    interval_end_dt.loc[is_2400] = interval_end_dt.loc[is_2400] + pd.Timedelta(days=1)

    # Check for any parsing errors
    bad_rows = interval_end_dt.isna()
    if bad_rows.any():
        print(f"Found and dropped rows with problematic date/time strings.")
        combined_df = combined_df[~bad_rows]
        interval_end_dt = interval_end_dt.dropna()

    if combined_df.empty:
        print("No valid data remaining after time conversion. Exiting.")
        return
        
    # The final datetime is the *start* of the interval (end time - 15 mins)
    combined_df['datetime'] = interval_end_dt - pd.Timedelta(minutes=15)

    # 7. Pivot
    print("Pivoting DataFrame...")
    final_df = combined_df.pivot_table(
        index='datetime', 
        columns='Fuel', 
        values='Generation_MW', 
        aggfunc='sum'
    )
    
    # 8. Clean and Save
    final_df = final_df.sort_index()
    final_df = final_df.fillna(0) # Assume missing values are 0 generation
    output_csv = 'ercot_generation_by_fuel_2023_timeseries_from_excel.csv'
    final_df.to_csv(output_csv)
    
    print("\n--- Success! ---")
    print(f"Successfully processed data and saved to: {output_csv}")
    print("\nDataFrame Head:")
    print(final_df.head())
    print("\nDataFrame Info:")
    final_df.info()

    return final_df

In [7]:
df = process_excel_file("FuelMixReport_PreviousYears/IntGenbyFuel2023.xlsx")

Reading all worksheets from FuelMixReport_PreviousYears/IntGenbyFuel2023.xlsx...
Processing 12 monthly worksheets...
Combining data...
Creating datetime index...
Pivoting DataFrame...

--- Success! ---
Successfully processed data and saved to: ercot_generation_by_fuel_2023_timeseries_from_excel.csv

DataFrame Head:
Fuel                  Biomass        Coal         Gas       Gas-CC     Hydro  \
datetime                                                                       
2022-12-31 23:45:00  4.171277  895.728246  122.363679  1263.459487  1.093971   
2023-01-01 00:00:00  4.185436  921.604654  164.266324  1401.927280  4.053273   
2023-01-01 00:15:00  4.182232  893.081975  162.040533  1338.986303  5.952185   
2023-01-01 00:30:00  4.184353  881.553732  158.855930  1295.088153  6.008958   
2023-01-01 00:45:00  4.183658  863.928866  158.186564  1280.607814  5.877862   

Fuel                     Nuclear     Other  Solar        WSL         Wind  
datetime                                      

In [8]:
df

Fuel,Biomass,Coal,Gas,Gas-CC,Hydro,Nuclear,Other,Solar,WSL,Wind
datetime,,,,,,,,,,
2022-12-31 23:45:00,4.171277,895.728246,122.363679,1263.459487,1.093971,1275.899223,2.534023,0.000000,-22.782410,5779.415217
2023-01-01 00:00:00,4.185436,921.604654,164.266324,1401.927280,4.053273,1277.477718,1.431746,0.000000,-29.810096,5389.258859
2023-01-01 00:15:00,4.182232,893.081975,162.040533,1338.986303,5.952185,1277.697111,6.522572,0.000000,-2.712604,5398.169670
2023-01-01 00:30:00,4.184353,881.553732,158.855930,1295.088153,6.008958,1277.609374,6.853462,0.000000,-6.145239,5439.582984
2023-01-01 00:45:00,4.183658,863.928866,158.186564,1280.607814,5.877862,1277.783076,5.005813,0.000000,-10.130714,5458.294312
...,...,...,...,...,...,...,...,...,...,...
2023-12-31 22:30:00,3.220178,1486.670716,193.412576,3140.939980,0.908707,1274.656766,22.339712,0.002553,-13.852292,4456.710086
2023-12-31 22:45:00,3.219507,1489.183959,202.343671,3300.058828,0.909911,1275.237806,33.670299,0.002278,-9.995207,4281.815161
2023-12-31 23:00:00,3.226354,1488.438393,205.569248,3392.353081,0.910344,1275.469833,36.317436,0.003548,-6.061843,4120.181071
